# CE Page Viewer

Simple notebook to view CE document pages for manual review and exploration.

In [1]:
import pandas as pd
from pathlib import Path

# Display settings
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)

## Load Data

In [2]:
# Load CE documents
documents = pd.read_parquet('../data/analysis/documents_combined.parquet')
ce_documents = documents[documents["dataset_source"] == "CE"]
print(f"Loaded {len(ce_documents):,} CE documents")

# Load CE pages
ce_pages = pd.read_parquet('../data/processed/ce/pages.parquet')
print(f"Loaded {len(ce_pages):,} CE pages")

Loaded 73,544 CE documents
Loaded 73,544 CE pages


## Helper Functions

In [3]:
def list_documents(project_id):
    """List all documents for a given project."""
    docs = ce_documents[ce_documents["project_id"] == project_id]
    if len(docs) == 0:
        print(f"No documents found for project_id: {project_id}")
        return None
    
    print(f"\nDocuments for project: {project_id}")
    print("-" * 80)
    display_cols = ["document_id", "document_type", "document_type_category", 
                    "main_document", "file_name", "total_pages"]
    available_cols = [c for c in display_cols if c in docs.columns]
    return docs[available_cols]

In [4]:
def view_page(document_id, page_num=1):
    """View a specific page from a document."""
    page = ce_pages[(ce_pages["document_id"] == document_id) & 
                    (ce_pages["page_number"] == page_num)]
    
    if len(page) == 0:
        print(f"Page {page_num} not found for document_id: {document_id}")
        return None
    
    # Get document metadata
    doc = ce_documents[ce_documents["document_id"] == document_id]
    if len(doc) > 0:
        print(f"\n{'='*80}")
        print(f"Document: {doc.iloc[0].get('file_name', 'N/A')}")
        print(f"Type: {doc.iloc[0].get('document_type', 'N/A')} | "
              f"Category: {doc.iloc[0].get('document_type_category', 'N/A')} | "
              f"Main: {doc.iloc[0].get('main_document', 'N/A')}")
        print(f"{'='*80}")
    
    print(f"\n--- PAGE {page_num} ---\n")
    print(page.iloc[0]["page_text"])
    print(f"\n--- END PAGE {page_num} ---\n")
    return page.iloc[0]["page_text"]

In [5]:
def view_pages(document_id, start_page=1, end_page=5):
    """View a range of pages from a document."""
    # Get document metadata
    doc = ce_documents[ce_documents["document_id"] == document_id]
    if len(doc) > 0:
        print(f"\n{'='*80}")
        print(f"Document: {doc.iloc[0].get('file_name', 'N/A')}")
        print(f"Type: {doc.iloc[0].get('document_type', 'N/A')} | "
              f"Category: {doc.iloc[0].get('document_type_category', 'N/A')} | "
              f"Main: {doc.iloc[0].get('main_document', 'N/A')}")
        print(f"{'='*80}")
    
    for page_num in range(start_page, end_page + 1):
        page = ce_pages[(ce_pages["document_id"] == document_id) & 
                        (ce_pages["page_number"] == page_num)]
        if len(page) > 0:
            print(f"\n--- PAGE {page_num} ---\n")
            print(page.iloc[0]["page_text"])
            print(f"\n--- END PAGE {page_num} ---\n")
        else:
            print(f"Page {page_num} not found")
            break

In [23]:
def view_project_pages(project_id):
    """View all pages of each document in a project (no truncation)."""
    docs = ce_documents[ce_documents["project_id"] == project_id]
    
    if len(docs) == 0:
        print(f"No documents found for project_id: {project_id}")
        return
    
    for _, doc in docs.iterrows():
        doc_id = doc["document_id"]
        print(f"\n{'='*80}")
        print(f"Document: {doc.get('file_name', 'N/A')}")
        print(
            f"Type: {doc.get('document_type', 'N/A')} | "
            f"Category: {doc.get('document_type_category', 'N/A')} | "
            f"Main: {doc.get('main_document', 'N/A')}"
        )
        print(f"{'='*80}")
        
        doc_pages = (
            ce_pages[ce_pages["document_id"] == doc_id]
            .sort_values("page_number")
        )
        
        for _, page in doc_pages.iterrows():
            print(f"\n--- PAGE {page['page_number']} ---\n")
            print(page["page_text"])


In [7]:
def search_pages(project_id, search_term):
    """Search for a term in all pages of a project."""
    docs = ce_documents[ce_documents["project_id"] == project_id]
    doc_ids = docs["document_id"].tolist()
    
    matching_pages = ce_pages[
        (ce_pages["document_id"].isin(doc_ids)) & 
        (ce_pages["page_text"].str.contains(search_term, case=False, na=False))
    ]
    
    print(f"\nFound {len(matching_pages)} pages containing '{search_term}'")
    print("-" * 80)
    
    for _, page in matching_pages.iterrows():
        doc = docs[docs["document_id"] == page["document_id"]].iloc[0]
        print(f"\nDocument: {doc.get('file_name', 'N/A')} | Page: {page['page_number']}")
        
        # Show context around the search term
        text = page["page_text"]
        idx = text.lower().find(search_term.lower())
        if idx >= 0:
            start = max(0, idx - 200)
            end = min(len(text), idx + len(search_term) + 200)
            context = text[start:end]
            if start > 0:
                context = "..." + context
            if end < len(text):
                context = context + "..."
            print(context)
    
    return matching_pages

In [8]:
def get_random_project(n_docs=2):
    """Get a random project with at least n_docs documents."""
    import random
    doc_counts = ce_documents.groupby("project_id").size()
    eligible = doc_counts[doc_counts >= n_docs].index.tolist()
    
    project_id = random.choice(eligible)
    print(f"Random project with {doc_counts[project_id]} documents: {project_id}")
    return project_id

## Explore a specific project

Set the `project_id` below and run the cells to explore.

In [ ]:
# Set your project ID here
project_id = "2135a0a9-e30c-47a6-4781-d35b7eb72322"


In [18]:
# List all documents for this project
list_documents(project_id)


Documents for project: 2135a0a9-e30c-47a6-4781-d35b7eb72322
--------------------------------------------------------------------------------


,document_id,document_type,document_type_category,main_document,file_name,total_pages
83786,7b584951-905d-6435-abde-be943dec2ef0,CE,decision,YES,Wyarno Grazing Lease CX.pdf,8
83787,d7387aeb-510e-38ea-39c4-feeea4d87c77,CE,decision,YES,Wyarno Grazing Lease DR.pdf,3


In [24]:
# View first 3 pages of each document
#view_project_pages(project_id, max_pages_per_doc=3, truncate=1500)
view_project_pages(project_id)


Document: Wyarno Grazing Lease CX.pdf
Type: CE | Category: decision | Main: YES

--- PAGE 1-8 ---

U.S. Department of the Interior
Bureau of Land Management

Categorical Exclusion: Grazing Lease Renewal
Wyarno Allotment #12042, Lease #4907074
DOI-BLM-WY-P070-2018-133-CX

PREPARING OFFICE
U.S. Department of the Interior
Bureau of Land Management
Buffalo Field Office
Wyoming

A. Description of the Proposed Action
Background
The Bureau of Land Management (BLM), Buffalo Field Office (BFO) proposes to renew a 10 -
year grazing lease for the Wyarno allotment (#12042). The Wyarno allotment is about 8 miles
east of Sheridan Wyoming in Sheridan County. The allotment consists of about 2,100 total acres
of which 6% is BLM land and 94% is deeded land. The lease authorizing grazing on the
allotment includes a total of 120 acres of BLM land and 24 animal unit months (AUMs) of
forage. Grazing use is authorized for cattle and sheep from 3/1-2/28. The Wyarno allotment is a
"Custodial" allotment, meani

## View Specific Document

In [13]:
# Get document IDs for the project
docs = ce_documents[ce_documents["project_id"] == project_id]
docs[["document_id", "file_name", "main_document", "document_type"]]

,document_id,file_name,main_document,document_type
73227,65c6bb9f-ea78-d49b-3513-f5fc9c09da57,DOI-BLM-CO-N010-2021-0043-CX-DR_Road 1509 Emergency Repair_for web.pdf,NO,OTHER
73228,25334806-7f45-75fb-d4b0-cad8e1e5567a,DOI-BLM-CO-N010-2021-0043-CX_Road 1509 Emergency Repair_for web.pdf,YES,CE


In [14]:
# Set document_id and view specific pages
document_id = docs.iloc[0]["document_id"]  # First document
view_pages(document_id, start_page=1, end_page=5)


Document: DOI-BLM-CO-N010-2021-0043-CX-DR_Road 1509 Emergency Repair_for web.pdf
Type: OTHER | Category: other | Main: NO
Page 1 not found


## Search for Text

In [15]:
# Search for dates or specific text
search_pages(project_id, "2024")


Found 0 pages containing '2024'
--------------------------------------------------------------------------------


,document_id,page_number,page_text


In [16]:
# Search for decision-related text
search_pages(project_id, "approved")


Found 2 pages containing 'approved'
--------------------------------------------------------------------------------

Document: DOI-BLM-CO-N010-2021-0043-CX-DR_Road 1509 Emergency Repair_for web.pdf | Page: 1-4
...e emergency repair of BLM Road 1509 in Moffat County, Colorado,
near the Little Snake/White River Field Office boundary. The repair is expected to consist of
layering compatible rock material and BLM-approved Geotech material, then installing a culvert
and armoring the inlet and outlet of the culvert with local rock material from a nearby rock pile,
within BLM standards. The road repairs are expected to b...

Document: DOI-BLM-CO-N010-2021-0043-CX_Road 1509 Emergency Repair_for web.pdf | Page: 1-11
...safety.
Conformance with the Land Use Plan
The Proposed Action is subject to and is in conformance (43 CFR 1610.5) with the following
land use plan:
Land Use Plan: Little Snake Record of Decision and Approved Resource Management Plan
(ROD/RMP)
Date Approved: October 2011
Decisi

document_id page_number  \
4688  65c6bb9f-ea78-d49b-3513-f5fc9c09da57         1-4   
4689  25334806-7f45-75fb-d4b0-cad8e1e5567a        1-11   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                